# Modelos e Estratégias de Trading
## Ajustando um modelo VAR à PETR4

Uirá Caiado. 05 de Setembro, 2016

**Resumo**

*Uma das principais necessidades quando falamos trading é realizar algum tipo de previsão sobre o estado futuro do instrumento operado para que possamos posicionar nossas ofertas. Como um dos objetivos da análise de séries temporais multivariadas é a realização de forecasts, nesta atividade vou implementar um modelo desta classe chamado Vetor Autoregressivo (VAR). Vou ajustar o modelo aos retornos e informações de book da PETR4 e analisar a performance realizando um back-test em uma parte dos dados coletados que ainda não tenha sido utilizada.*

## 1. Introdução

Nesta sessão vou ...


### 1.1. Modelagem ...

bla bla

In [286]:
import pandas as pd
df = pd.read_excel('data/VAR3_inputs_e_outputs.xlsx', header=None)
l_index = pd.date_range(start= '01/01/2014', end='01/12/2016')
df.index = l_index[:df.shape[0]]

In [287]:
from statsmodels.tsa.api import VAR
import statsmodels.tsa.vector_ar.util as util
model = VAR(df)
results = model.fit(3)
aux = util.get_var_endog(df.values, 3)

In [288]:
import var_model.vector_autoregression as var
reload(var)
self = var.VectorAutoregression(df)
self.fit(3)

In [289]:
# error in Z
print "error in Z: {:0.8f}".format(sum(sum(abs(self.na_Z.T - aux))))

error in Z: 0.00000000


In [290]:
#print error
print "error in A: {:0.8f}".format(sum(sum(sum(abs(self.na_A - results.coefs)))))

error in A: 0.00000000


In [291]:
# print betas
print "error in B: {:0.8f}".format(sum(sum(abs(self.na_betahat.T - results.params.values))))

error in B: 9.87466903


In [292]:
print "error in residuals: {:0.8f}".format(sum(sum(abs(self.na_U.T - results.resid.values))))

error in residuals: 11.05554726


In [223]:
na_betas = self.na_betahat.copy()


In [224]:
for idx in xrange(len(na_betas)):
    na_betas[idx] = na_betas[idx].T

In [233]:
pd.DataFrame(na_betas).ix[:, 1:5]

,1,2,3,4,5
0,-0.043374,0.005321,0.008242,0.035804,-0.064091
1,0.128951,0.178780,0.115180,0.166959,-0.059743
2,-0.127479,0.003759,-0.050708,0.048654,-0.097786
3,0.049648,0.032509,0.007212,-0.117873,0.268141
4,-0.071250,-0.072283,-0.041946,-0.049864,-0.054190


In [215]:
results.params

,0,1,2,3,4
const,0.000005,0.000422,0.000810,-0.000025,-0.001338
L1.0,-0.043374,0.005321,0.008242,0.035804,-0.064091
L1.1,0.128951,0.178780,0.115180,0.166959,-0.059743
L1.2,-0.127479,0.003759,-0.050708,0.048654,-0.097786
L1.3,0.049648,0.032509,0.007212,-0.117873,0.268141
L1.4,-0.071250,-0.072283,-0.041946,-0.049864,-0.054190
L2.0,0.013431,-0.244118,-0.047898,-0.043729,-0.028104
L2.1,0.001495,-0.004717,0.179970,0.159101,0.096378
L2.2,-0.242662,-0.345717,-0.487353,-0.365092,-0.391956
L2.3,0.245055,0.413840,0.295174,0.193884,0.162951


In [234]:
self.na_Z.shape

(16, 197)

In [236]:
self.na_A.shape

(3, 5, 5)

In [261]:
pd.DataFrame(np.vstack([self.na_v, self.na_A.reshape((1, 15, 5))[0]]))

,0,1,2,3,4
0,0.000005,0.000422,0.000810,-0.000025,-0.001338
1,-0.043374,0.128951,-0.127479,0.049648,-0.071250
2,0.005321,0.178780,0.003759,0.032509,-0.072283
3,0.008242,0.115180,-0.050708,0.007212,-0.041946
4,0.035804,0.166959,0.048654,-0.117873,-0.049864
5,-0.064091,-0.059743,-0.097786,0.268141,-0.054190
6,0.013431,0.001495,-0.242662,0.245055,-0.075217
7,-0.244118,-0.004717,-0.345717,0.413840,-0.032270
8,-0.047898,0.179970,-0.487353,0.295174,0.055272
9,-0.043729,0.159101,-0.365092,0.193884,0.068610


In [142]:
results.resid.values[0]


array([-0.00192395,  0.00042494, -0.01179083,  0.00293511, -0.02745351])

In [152]:
self.na_U.T[0]

array([ 0.01645741, -0.00435309,  0.01869885, -0.02866829, -0.04331394])

In [149]:
pd.DataFrame(self.na_Sigma.T).to_clipboard()


In [153]:

self.na_Sigma - results.

ValueError: operands could not be broadcast together with shapes (5,5) (197,5) 

In [ ]:
        avobs = len(y_sample)

        df_resid = 197 - (model.neqs * 3 + 1)

        sse = np.dot(resid.T, resid)
        omega = sse / df_resid

In [155]:
results.sigma_u.to_clipboard()

In [156]:
model.neqs

5

In [157]:
df_resid = 197 - (model.neqs * 3 + 1)

In [158]:
df_resid

181

In [159]:
197 -5*3 -1

181

In [161]:
np.dot(self.na_U.T, self.na_U).shape

(197, 197)

In [172]:
results.resid.head()

,0,1,2,3,4
2014-01-04,-0.001924,0.000425,-0.011791,0.002935,-0.027454
2014-01-05,0.005155,0.008669,0.001139,0.008313,-0.003835
2014-01-06,0.004475,0.000608,0.010624,0.005673,-0.012449
2014-01-07,-0.003724,-0.016574,-0.013673,-0.019383,0.004235
2014-01-08,0.000772,-0.002265,-0.004111,0.002673,-0.005990


In [174]:
pd.DataFrame(self.na_y[3:].T - np.dot(self.na_betahat, self.na_Z)).T.head()

,0,1,2,3,4
0,0.016457,-0.004353,0.018699,-0.028668,-0.043314
1,0.004348,0.013696,-0.028101,0.031078,-0.001872
2,0.010514,0.012866,0.002784,0.018771,0.001543
3,-0.007361,-0.025649,0.006894,-0.022466,0.004840
4,0.001334,-0.011118,0.008890,-0.020785,-0.019724


In [176]:
np.dot(aux, results.params).shape

(197, 5)

In [185]:
sum(abs(np.dot(self.na_Z.T, self.na_betahat.T) - np.dot(aux, results.params)))

array([ 1.1127088 ,  1.64890783,  4.07310311,  2.94496785,  1.27585967])

(197, 16)

In [191]:
sum(sum(self.na_Z.T - aux))

0.0

(16, 5)

In [201]:
sum(abs(self.na_betahat.T - results.params.values))

array([ 1.567226  ,  1.80978305,  2.657704  ,  2.09329362,  1.74666236])

In [109]:
self.na_Z = self._get_z_mat(i_p)  # Z := [Z_0, ..., Z_T] (T x (Kp+1))
na_Z = self.na_Z.copy()  # ((Kp+1) x T)
na_ysample = self.na_y[i_p:].T  # Y := (y_1, ..., y_T)  (K x T)
na_betahat = self.na_betahat

In [115]:
np.dot(na_Z.T, na_betahat.T)  # (T x K)

array([[ -1.58788099e-02,   1.01057442e-02,  -3.92745603e-02,
          2.43942848e-02,   6.49996384e-03],
       [ -3.19178401e-03,  -2.28862060e-03,   2.47514605e-02,
         -2.76183928e-02,   1.87156710e-03],
       [  4.39659245e-03,  -3.45903852e-03,   1.92445039e-02,
         -6.00896119e-03,  -3.10632844e-03],
       [  1.07966472e-03,   1.05546001e-02,  -1.31618076e-02,
          9.19900186e-03,  -3.27578766e-03],
       [ -4.77706781e-03,  -1.12350475e-03,  -1.96515661e-02,
          1.72845251e-02,   2.38665655e-03],
       [  1.16712260e-03,  -5.37820904e-03,   1.74498773e-02,
         -1.99538461e-02,   1.90829759e-03],
       [  7.05262071e-03,   8.18829839e-03,   1.74666789e-02,
         -1.03703092e-02,  -6.71222591e-03],
       [ -8.04487302e-03,   4.73255231e-03,  -3.54638349e-02,
          2.85991814e-02,   4.29448028e-04],
       [ -2.16832871e-03,  -1.50591493e-03,   1.70605372e-02,
         -1.16175239e-02,  -4.49673682e-03],
       [  6.33373798e-04,   7.0167013

In [117]:
na_ysample.shape

(5, 197)

In [113]:
na_betahat.shape

(5, 16)

In [122]:
na_U = na_ysample - np.dot(na_betahat, na_Z)

In [124]:
np.dot(na_U, na_U.T).shape

(5, 5)

## 2. Implementando o Modelo

Nesta sessão detalharemos e implementaremos ...

### 2.1. Bla

bla bla


## 4. Conclusão

bla bla




## 5. Últimas Considerações

bla bla


*Style notebook and change matplotlib defaults*

In [5]:
#loading style sheet
from IPython.core.display import HTML
HTML(open('ipython_style.css').read())

In [6]:
#changing matplotlib defaults
%matplotlib inline
import seaborn as sns
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("Set2", 10))